# Qwen3-14B LoRA 파인튜닝 with Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048

fourbit_models = [
    "unsloth/Qwen3-1.7B",  # Qwen 14B 보다 2배 이상 빠름
    "unsloth/Qwen3-4B",
    "unsloth/Qwen3-8B",
    "unsloth/Qwen3-14B",
    "unsloth/Qwen3-32B",
    # 뛰어난 정확도와 낮은 메모리 사용을 위한 4bit dynamic quants
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",
]  # 더 많은 모델: https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen3-14B",
    max_seq_length=max_seq_length,  # 컨텍스트 길이 - 더 길 수 있지만 더 많은 메모리 사용
    load_in_4bit=True,  # 4bit는 메모리를 훨씬 적게 사용
    load_in_8bit=False,  # 조금 더 정확하지만, 2배 메모리 사용
    full_finetuning=False,  # 이제 full finetuning 지원!
    # token = "hf_...",      # gated 모델 사용 시 필요
)

이제 LoRA 어댑터를 추가하여 전체 파라미터의 1~10%만 업데이트하면 됩니다!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # 0보다 큰 아무 숫자나 선택! 권장: 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # 긴 컨텍스트를 위해 True 또는 "unsloth" 사용
    random_state=3407,
    use_rslora=False,  # rank stabilized LoRA 지원
    loftq_config=None,  # LoftQ 지원
)

<a name="Data"></a>
### 데이터 준비
Qwen3는 추론 모드와 비추론 모드를 모두 가지고 있습니다. 따라서 2개의 데이터셋을 사용해야 합니다:

1. [AIMO](https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2/leaderboard) (AI Mathematical Olympiad - Progress Prize 2) 챌린지에서 우승하는데 사용된 [Open Math Reasoning]() 데이터셋을 사용합니다! DeepSeek R1을 사용하고 95% 이상의 정확도를 얻은 검증 가능한 추론 추적의 10%를 샘플링합니다.

2. ShareGPT 스타일의 [Maxime Labonne의 FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) 데이터셋도 활용합니다.  
하지만 이를 HuggingFace의 일반 멀티턴 형식으로 변환해야 합니다.

In [ ]:
from datasets import load_dataset

reasoning_dataset = load_dataset("unsloth/OpenMathReasoning-mini", split="cot")
non_reasoning_dataset = load_dataset("mlabonne/FineTome-100k", split="train")

두 데이터셋의 구조를 확인해봅시다:

In [ ]:
reasoning_dataset

In [ ]:
non_reasoning_dataset

이제 추론 데이터셋을 대화 형식으로 변환합니다:

In [ ]:
def generate_conversation(examples):
    problems = examples["problem"]
    solutions = examples["generated_solution"]
    conversations = []
    for problem, solution in zip(problems, solutions, strict=True):
        conversations.append(
            [
                {"role": "user", "content": problem},
                {"role": "assistant", "content": solution},
            ]
        )
    return {
        "conversations": conversations,
    }

In [ ]:
reasoning_conversations = tokenizer.apply_chat_template(
    reasoning_dataset.map(generate_conversation, batched=True)["conversations"],
    tokenize=False,
)

변환된 첫 번째 행을 확인해봅시다:

In [ ]:
reasoning_conversations[0]

다음으로 비추론 데이터셋을 가져와서 대화 형식으로 변환합니다.

먼저 Unsloth의 `standardize_sharegpt` 함수를 사용하여 데이터셋의 형식을 수정해야 합니다.

In [ ]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(non_reasoning_dataset)

non_reasoning_conversations = tokenizer.apply_chat_template(
    dataset["conversations"],
    tokenize=False,
)

첫 번째 행을 확인해봅시다

In [ ]:
non_reasoning_conversations[0]

이제 두 데이터셋의 길이를 확인해봅시다:

In [ ]:
print(len(reasoning_conversations))
print(len(non_reasoning_conversations))

비추론 데이터셋이 훨씬 더 깁니다. 모델이 일부 추론 능력을 유지하면서도 특별히 채팅 모델을 원한다고 가정해봅시다.

채팅 전용 데이터의 비율을 정의합시다. 목표는 두 데이터셋의 혼합을 정의하는 것입니다.

75% 추론과 25% 채팅 기반을 선택해봅시다:

In [ ]:
chat_percentage = 0.25

추론 데이터셋을 75% (또는 100% - chat_percentage)로 샘플링합시다

In [ ]:
import pandas as pd

non_reasoning_subset = pd.Series(non_reasoning_conversations)
non_reasoning_subset = non_reasoning_subset.sample(
    int(len(reasoning_conversations) * (chat_percentage / (1 - chat_percentage))),
    random_state=2407,
)
print(len(reasoning_conversations))
print(len(non_reasoning_subset))
print(len(non_reasoning_subset) / (len(non_reasoning_subset) + len(reasoning_conversations)))

마지막으로 두 데이터셋을 결합합니다:

In [ ]:
data = pd.concat([pd.Series(reasoning_conversations), pd.Series(non_reasoning_subset)])
data.name = "text"

from datasets import Dataset

combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed=3407)

<a name="Train"></a>
### 모델 훈련
이제 모델을 훈련해봅시다. 속도를 높이기 위해 60 스텝을 수행하지만, 전체 실행을 위해 `num_train_epochs=1`로 설정하고 `max_steps=None`로 끌 수 있습니다.

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=combined_dataset,
    eval_dataset=None,  # 평가 설정 가능!
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # 배치 사이즈를 모방하기 위해 GA 사용!
        warmup_steps=5,
        # num_train_epochs = 1, # 전체 1회 훈련 실행을 위해 이것을 설정하세요.
        max_steps=30,
        learning_rate=2e-4,  # 긴 훈련 실행을 위해 2e-5로 감소
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.001,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",  # TrackIO/WandB 등 사용
    ),
)

In [ ]:
# @title 현재 메모리 통계 표시
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. 최대 메모리 = {max_memory} GB.")
print(f"{start_gpu_memory} GB 메모리 예약됨.")

모델을 훈련해봅시다! 훈련 실행을 재개하려면 `trainer.train(resume_from_checkpoint = True)`로 설정하세요

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title 최종 메모리 및 시간 통계 표시
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} 초 훈련에 사용됨.")
print(f"{round(trainer_stats.metrics['train_runtime'] / 60, 2)} 분 훈련에 사용됨.")
print(f"최대 예약 메모리 = {used_memory} GB.")
print(f"훈련을 위한 최대 예약 메모리 = {used_memory_for_lora} GB.")
print(f"최대 메모리의 최대 예약 메모리 % = {used_percentage} %.")
print(f"최대 메모리의 훈련용 최대 예약 메모리 % = {lora_percentage} %.")

<a name="Inference"></a>
### 추론
Unsloth 네이티브 추론을 통해 모델을 실행해봅시다! `Qwen-3` 팀에 따르면, 추론 인퍼런스를 위한 권장 설정은 `temperature = 0.6, top_p = 0.95, top_k = 20`입니다.

일반 채팅 기반 추론의 경우, `temperature = 0.7, top_p = 0.8, top_k = 20`입니다.

In [ ]:
messages = [{"role": "user", "content": "Solve (x + 2)^2 = 0."}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,  # 생성을 위해 반드시 추가
    enable_thinking=False,  # 사고 비활성화
)

from transformers import TextStreamer

_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    max_new_tokens=256,  # 더 긴 출력을 위해 증가!
    temperature=0.7,
    top_p=0.8,
    top_k=20,  # 비사고용
    streamer=TextStreamer(tokenizer, skip_prompt=True),
)

In [ ]:
messages = [{"role": "user", "content": "Solve (x + 2)^2 = 0."}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,  # Thinking 생성을 위해 반드시 추가
    enable_thinking=True,  # Thinking 활성화
)

from transformers import TextStreamer

_ = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    max_new_tokens=1024,  # 더 긴 출력을 위해 증가!
    temperature=0.6,
    top_p=0.95,
    top_k=20,  # Thinking 용도
    streamer=TextStreamer(tokenizer, skip_prompt=True),
)

<a name="Save"></a>
### 파인튜닝된 모델 저장 및 로드
최종 모델을 LoRA 어댑터로 저장하려면, 온라인 저장을 위해 Huggingface의 `push_to_hub`를 사용하거나 로컬 저장을 위해 `save_pretrained`를 사용하세요.

이 코드는 LoRA 어댑터만 저장하며, 전체 모델은 저장하지 않습니다. 16bit 또는 GGUF로 저장하려면 아래쪽 셀을 참고하세요.

In [ ]:
model.save_pretrained("lora_model")  # 로컬 저장
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # 온라인 저장
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # 온라인 저장

이제 추론을 위해 방금 저장한 LoRA 어댑터를 로드하려면, `False`를 `True`로 설정하세요:

In [ ]:
if False:
    from unsloth import FastLanguageModel

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="lora_model",  # 훈련된 모델
        max_seq_length=2048,
        load_in_4bit=True,
    )

### 16bit로 저장
또한 모든 어댑터를 기본 모델에 완전히 병합하고, 모델 가중치를 16비트 또는 4비트로 저장할 수 있습니다. 이를 Hugging Face에 온라인으로 푸시하려면 Hugging Face 토큰을 받아야 합니다.

**[참고]** 16bit 모델이 마음에 들면 quantize_to_gguf 셀의 방법을 선택하세요. 네이티브 GGUF 추론을 위해 `quantization_method = "f16"`을 설정할 수 있습니다.

In [ ]:
if False:
    model.save_pretrained_merged("model", tokenizer, save_method="merged_16bit")
    # model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

### 4bit로 저장
더 저렴한 추론을 위해 4bit로 병합하여 저장할 수도 있습니다.

**[참고]** 4bit 모델이 마음에 들면 quantize_to_gguf 셀의 방법을 선택하세요. 네이티브 GGUF 추론을 위해 `quantization_method = "q4_k_m"`을 설정할 수 있습니다.

In [ ]:
if False:
    model.save_pretrained_merged("model", tokenizer, save_method="merged_4bit")
    # model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

### llama.cpp로 내보내기 (GGUF) 또는 vLLM 추론
또는 llama.cpp 또는 Mac, Windows, Linux를 위한 Ollama에서 사용하기 위해 GGUF로 직접 내보낼 수 있습니다. 또는 vLLM에서 사용하기 위해 sharded 모델로 내보낼 수 있습니다.

최적의 결과를 위해 `quantization_method` 인자 - `"q4_k_m"`, `"q5_k_m"`, `"f16"` 등에 대한 llama.cpp 커뮤니티 가이드를 따르는 것이 권장됩니다!

**[참고]** vLLM에 `"lora"`를 푸시하려면, 먼저 `"lora"`가 만들어지도록 해야 합니다!

In [ ]:
if False:
    # lora 어댑터만 저장 - vLLM에서 사용 가능
    model.save_pretrained("lora_model")
    tokenizer.save_pretrained("lora_model")

    # 병합하고 quantization_method로 GGUF 또는 기타 형식으로 저장
    model.save_pretrained_gguf("model", tokenizer, quantization_method="q4_k_m")
    # model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

    # vLLM에서 사용할 sharded 모델로 저장
    # model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
    # model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")